In [1]:
from pynucastro.rates import Library, Nucleus, RateFilter
from pynucastro.nucdata import BindingTable
from pynucastro.networks import PythonNetwork, Composition

In [2]:
full_lib = Library("reaclib-2017-10-20")

In [3]:
# Could introduce pp-chain nuclei (d, t, he3, be7, li7, etc.)

core_nuclei = ["p", "d", "he3", "he4", "li7", "be7", "be8", "b8", "c12",
               "n13", "n14", "n15", "o14", "o15", "o16", "o17", "o18",
               "f17", "f18", "f19", "f20", "ne18", "ne19", "ne20", "ne21"]
core_nuclei = list(map(Nucleus, core_nuclei))

In [4]:
core_lib = full_lib.linking_nuclei(core_nuclei)

In [5]:
def is_beta_plus(rate):
    """ Filter for beta+ decays (and electron captures). """
    
    if len(rate.products) != len(rate.reactants):
        return False
    if len(rate.reactants) != 1:
        return False
    
    react, = rate.reactants
    prod, = rate.products
    return prod.Z < react.Z

In [6]:
# Restrict the library to these 7 reaction types

# Forward rates
p_gamma = RateFilter(reactants="p", max_products=1, exact=False)
alpha_gamma = RateFilter(reactants="he4", max_products=1, exact=False)
alpha_p = RateFilter(reactants="he4", products="p", exact=False)

# Reverse and weak rates
gamma_p = RateFilter(products="p", max_reactants=1, exact=False)
gamma_alpha = RateFilter(products="he4", max_reactants=1, exact=False)
p_alpha = RateFilter(reactants="p", products="he4", exact=False)
beta_plus = RateFilter(filter_function=is_beta_plus)

# Compute reduced library
red_lib = full_lib.filter((p_gamma, alpha_gamma, alpha_p,
        gamma_p, gamma_alpha, p_alpha, beta_plus))

In [7]:
# Make BindingTable a dict for easy containment checking
# Perhaps we could modify the BindingTable class to use one?
# That can be done without changing the interface
bintable = BindingTable()
nuclides = {(nuc.n, nuc.z): nuc for nuc in bintable.nuclides}

In [8]:
from collections import deque

def flatten(iterable):
    """ Take iterable of iterables, and flatten it to one dimension. """
    
    for col in iterable:
        
        for item in col:
            
            yield item
            
def append_all(q, iterable):
    """ Append all items in the iterable to the queue. """
    
    for item in iterable: q.append(item)
        
def product_limiter():
    """
    This helps trim the library a bit by excluding rates with
    products with more protons than Ni, heavier than Ni56,
    or with relatively high / low neutron percentages. 
    """
    
    # Proton number bounds
    Zlo, Zhi = 6, 52
    # Nucleon number bounds
    Alo, Ahi = 12, 108
    # Bounds on A / Z ratio to drop peripheral nuclei
    Rlo, Rhi = 1.69, 2.2
    
    def limit_products(r):
        
        meet_conds = \
        (
            (Zlo <= p.Z <= Zhi and
            Alo <= p.A <= Ahi and
            Rlo <= p.A / p.Z <= Rhi and
            (p.N, p.Z) in nuclides)
            or (p.Z, p.A) == (1, 1)
            or (p.Z, p.A) == (2, 4)
            for p in r.products
        )
        return all(meet_conds)
    
    return limit_products

In [9]:
limiter = product_limiter()
final_lib = Library(rates=core_lib.get_rates())

seeds = [nuc for nuc in core_nuclei if nuc.A >= 12]
encountered = set(seeds) | {Nucleus("p"), Nucleus("he4")}
seeds = deque(seeds)

while seeds:
    
    # Get the new rates with seed as a reactant
    seed = seeds.popleft()
    filt = RateFilter(reactants=seed, filter_function=limiter, exact=False)
    new_lib = red_lib.filter(filt)
    if new_lib is None: continue
    final_lib += new_lib
    
    # Append all unseen nuclei to the queue
    prod = (r.products for r in new_lib.get_rates())
    prod = flatten(prod)
    prod = filter(lambda p: p not in encountered, prod)
    prod = sorted(set(prod))
    append_all(seeds, prod)
    encountered.update(prod)
    
encountered = sorted(encountered)

In [10]:
final_lib

be7 --> li7    (be7 --> li7 <ec_reaclib_weak_>)
b8 --> be8    (b8 --> be8 <wc17_reaclib_weak_>)
o14 --> n14    (o14 --> n14 <wc12_reaclib_weak_>)
o15 --> n15    (o15 --> n15 <wc12_reaclib_weak_>)
f17 --> o17    (f17 --> o17 <wc12_reaclib_weak_>)
f18 --> o18    (f18 --> o18 <wc12_reaclib_weak_>)
f20 --> ne20    (f20 --> ne20 <wc12_reaclib_weak_>)
ne18 --> f18    (ne18 --> f18 <wc12_reaclib_weak_>)
ne19 --> f19    (ne19 --> f19 <wc12_reaclib_weak_>)
he3 --> p + d    (he3 --> p + d <de04_reaclib__reverse>)
he4 --> d + d    (he4 --> d + d <nacr_reaclib__reverse>)
be7 --> he4 + he3    (be7 --> he4 + he3 <cd08_reaclib__reverse>)
b8 --> p + be7    (b8 --> p + be7 <nacr_reaclib__reverse>)
b8 --> he4 + he4    (b8 --> he4 + he4 <wc12_reaclib_weak_>)
n13 --> p + c12    (n13 --> p + c12 <ls09_reaclib__reverse>)
o14 --> p + n13    (o14 --> p + n13 <lg06_reaclib__reverse>)
o15 --> p + n14    (o15 --> p + n14 <im05_reaclib__reverse>)
o16 --> p + n15    (o16 --> p + n15 <li10_reaclib__reverse>)
o16 --

In [11]:
encountered

[p,
 he4,
 c12,
 c13,
 n12,
 n13,
 n14,
 n15,
 o14,
 o15,
 o16,
 o17,
 o18,
 f16,
 f17,
 f18,
 f19,
 f20,
 ne17,
 ne18,
 ne19,
 ne20,
 ne21,
 ne22,
 na19,
 na20,
 na21,
 na22,
 na23,
 na24,
 mg21,
 mg22,
 mg23,
 mg24,
 mg25,
 mg26,
 al22,
 al23,
 al24,
 al25,
 al26,
 al27,
 al28,
 si24,
 si25,
 si26,
 si27,
 si28,
 si29,
 si30,
 p26,
 p27,
 p28,
 p29,
 p30,
 p31,
 p32,
 p33,
 s28,
 s29,
 s30,
 s31,
 s32,
 s33,
 s34,
 s35,
 cl29,
 cl30,
 cl31,
 cl32,
 cl33,
 cl34,
 cl35,
 cl36,
 cl37,
 ar31,
 ar32,
 ar33,
 ar34,
 ar35,
 ar36,
 ar37,
 ar38,
 ar39,
 k33,
 k34,
 k35,
 k36,
 k37,
 k38,
 k39,
 k40,
 k41,
 ca34,
 ca35,
 ca36,
 ca37,
 ca38,
 ca39,
 ca40,
 ca41,
 ca42,
 ca43,
 ca44,
 sc36,
 sc37,
 sc38,
 sc39,
 sc40,
 sc41,
 sc42,
 sc43,
 sc44,
 sc45,
 sc46,
 ti38,
 ti39,
 ti40,
 ti41,
 ti42,
 ti43,
 ti44,
 ti45,
 ti46,
 ti47,
 ti48,
 v40,
 v41,
 v42,
 v43,
 v44,
 v45,
 v46,
 v47,
 v48,
 v49,
 v50,
 cr42,
 cr43,
 cr44,
 cr45,
 cr46,
 cr47,
 cr48,
 cr49,
 cr50,
 cr51,
 cr52,
 mn44,
 mn45,
 mn46,

In [12]:
rp_net = PythonNetwork(libraries=[final_lib])

In [13]:
# rp_net.write_network()

In [14]:
comp = Composition(rp_net.get_nuclei())
comp.set_solar_like()

rates = rp_net.evaluate_rates(rho=1e6, T=1e9, composition=comp)